In [1]:
# 1. 라이브러리 설정
from calendar import c
from http.cookies import BaseCookie
import pandas as pd
import numpy as np

# matplotlib 2D, 3D
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

# Data Scaler
from sklearn.preprocessing import MinMaxScaler

# Data Shuffle
from sklearn.utils import shuffle

# reduce dimention 2D / 3D
from sklearn.manifold import TSNE
# from umap import UMAP

# 모델
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax
from tensorflow.keras.layers import Dense, Concatenate, Activation

# classification
from sklearn.cluster import KMeans


# 시드값 고정
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

ImportError: cannot import name 'find_stack_level' from 'pandas.util._exceptions' (C:\Users\kwonk\miniconda3\envs\kwon\lib\site-packages\pandas\util\_exceptions.py)

In [ ]:
def load_data(data_name):
    if data_name == 'lens_df_1M':
        # lens_df = pd.read_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/lens_all_mapping.csv').drop(columns=['Unnamed: 0', 'index'], axis=1)
        lens_df = pd.read_csv('../input/lens-mapping/lens_all_mapping.csv').drop(columns=['Unnamed: 0', 'index'], axis=1)

        #데이터 userid, content_id mapping
        user_id_mapping = {id:i for i, id in enumerate(lens_df['userId'].unique())}
        movie_id_mapping = {id:i for i, id in enumerate(lens_df['movieId'].unique())}

        lens_df['userId'] = lens_df['userId'].map(user_id_mapping)
        lens_df['movieId'] = lens_df['movieId'].map(movie_id_mapping)

        lens_df = lens_df.replace({'rating': {1:0, 2:0, 3:0, 4:1, 5:1}})
        genre_list = ['Action', 'Documentary', 'Horror', 'Romance']
        lens_df_genre_1 = pd.DataFrame()
        for i in genre_list:
            genre_df = lens_df[lens_df['genres'] == i]
            lens_df_genre_1 = pd.concat([lens_df_genre_1, genre_df], axis=0)
        lens_df = lens_df_genre_1.sample(frac=1).reset_index().drop(columns=['index'],axis=1)

       
        return lens_df

    elif data_name == 'lens_df_100K':
        lens_df = pd.read_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/lens_100k_all_mapping.csv').drop(columns=['Unnamed: 0'], axis=1)
        lens_df['userId'] = lens_df['userId'] -1
        lens_df['movieId'] = lens_df['movieId'] -1

        #데이터 userid, content_id mapping
        user_id_mapping = {id:i for i, id in enumerate(lens_df['userId'].unique())}
        movie_id_mapping = {id:i for i, id in enumerate(lens_df['movieId'].unique())}

        lens_df['userId'] = lens_df['userId'].map(user_id_mapping)
        lens_df['movieId'] = lens_df['movieId'].map(movie_id_mapping)
        
        return lens_df

In [ ]:
# genre one-hot encoding
def genre_onehot(all):
    genre_array = []
    for i in range(len(all)): 
      result_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      value = lens_df.at[i, 'genres']
      genres = value.split('|')
      for j in range(len(genres)):
          if genres[j] == 'Action':
              result_list[0] += 1
          elif genres[j] == 'Adventure':
              result_list[1] += 1
          elif genres[j] == 'Animation':
              result_list[2] += 1
          elif genres[j] == 'Children\'s':
              result_list[3] += 1
          elif genres[j] == 'Comedy':
              result_list[4] += 1                
          elif genres[j] == 'Crime':
              result_list[5] += 1
          elif genres[j] == 'Documentary':             
              result_list[6] += 1   
          elif genres[j] == 'Drama':   
              result_list[7] += 1             
          elif genres[j] == 'Fantasy':             
              result_list[8] += 1   
          elif genres[j] == 'Film-Noir':       
              result_list[9] += 1         
          elif genres[j] == 'Horror':              
              result_list[10] += 1  
          elif genres[j] == 'Musical':  
              result_list[11] += 1              
          elif genres[j] == 'Mystery':   
              result_list[12] += 1             
          elif genres[j] == 'Romance':      
              result_list[13] += 1          
          elif genres[j] == 'Sci-Fi':                
              result_list[14] += 1
          elif genres[j] == 'Thriller':                
              result_list[15] += 1
          elif genres[j] == 'War':                
              result_list[16] += 1
          elif genres[j] == 'Western':
              result_list[17] += 1
      genre_array.append(result_list)
    return genre_array

    # genre one-hot encoding
def genre_4_onehot(all):
    genre_array = []
    for i in range(len(all)): 
      result_list = [0,0,0,0]
      value = lens_df.at[i, 'genres']
      genres = value.split('|') 
      for j in range(len(genres)):
          if genres[j] == 'Action':
              result_list[0] += 1
          elif genres[j] == 'Documentary':
              result_list[1] += 1
          elif genres[j] == 'Horror':
              result_list[2] += 1
          elif genres[j] == 'Romance':
              result_list[3] += 1
      genre_array.append(result_list)
    return genre_array

# genre one-hot encoding
def gender_onehot(all):
    gender_array = []
    for i in range(len(all)): 
      result_list = [0,0]
      genders = lens_df.at[i, 'gender']
    #   genders = value.split('|')
      for j in range(len(genders)):
          if genders[j] == 'M':
              result_list[0] += 1
          elif genders[j] == 'F':
              result_list[1] += 1

      gender_array.append(result_list)
    return gender_array




# age one-hot encoding
def age_onehot(all):
    age_array = []
    for i in range(len(all)):
        if all.at[i, 'age'] == '1':
            age_array.append([1,0,0,0,0,0,0])
        elif all.at[i, 'age'] == '18':
            age_array.append([0,1,0,0,0,0,0])
        elif all.at[i, 'age'] == '25':
            age_array.append([0,0,1,0,0,0,0])
        elif all.at[i, 'age'] == '35':
            age_array.append([0,0,0,1,0,0,0])
        elif all.at[i, 'age'] == '45':
            age_array.append([0,0,0,0,1,0,0])
        elif all.at[i, 'age'] == '50':
            age_array.append([0,0,0,0,0,1,0])
        elif all.at[i, 'age'] == '56':
            age_array.append([0,0,0,0,0,0,1])
    return age_array

In [ ]:
def deep_array_model(raw_data, genre, TRAIN_SIZE, K):
        TRAIN_SIZE = 0.8
        # 이상한 점
        #ratings = shuffle(raw_data) # 섞여있는데 다시 섞는 부분 # array랑 순서가 맞지 않음
        cutoff = int(TRAIN_SIZE * len(raw_data))
        ratings_train = raw_data.iloc[:cutoff]
        ratings_test = raw_data.iloc[cutoff:]

        genre_arr_train = np.array(genre[:cutoff])
        genre_arr_test = np.array(genre[cutoff:])

        # gender_arr_train = np.array(gender[:cutoff])
        # gender_arr_test = np.array(gender[cutoff:])

        # age_arr_train = np.array(age[:cutoff])
        # age_arr_test = np.array(age[cutoff:])

        K = 200                             # Latent factor 수 
        #mu = raw_data.rating.mean()    # 전체 평균 

        M = raw_data.userId.max()+1       # Number of users
        N = raw_data.movieId.max()+1  # Number of movies

        # ##### (2)
        # Keras model
        # gender_input = Input(shape=(2,), name="gender_input")
        # age = Input(shape=(6, ), name="age_input")
        user_input = Input(shape=(1,), name="user_input")
        movie_input = Input(shape=(1,), name="movie_input")
        genre_input = Input(shape=(4,), name='genre_input')


        ## 이상한점 : feature 임베딩 layer 제대로 처리되지 않은점 : 차원 확대 부분에서...
        # G_embedding = Embedding(M, K, embeddings_regularizer=l2(), name="gender_embedding")(gender)
        p_embedding = Embedding(M, K, embeddings_regularizer=l2(), name="user_embedding")(user_input)
        Q_embedding = Embedding(N, K, embeddings_regularizer=l2(), name="movie_embedding")(movie_input)
        # gender_bias = Embedding(M, 1, embeddings_regularizer=l2())(gender)
        # item_bias = Embedding(N, 1, embeddings_regularizer=l2())(item)

        # G_embedding = Embedding(len(lens_df['gender'].unique()),K, embeddings_regularizer=l2(), name="gender_embedding")(gender_input)
        # gender_layer = Flatten()(G_embedding)
        genre_embedding = Embedding(len(genre[2]), K, embeddings_regularizer=l2(), name="genre_embedding")(genre_input)
        genre_layer = Flatten()(genre_embedding)
        # gender_embedding = Embedding(len(lens_df['genres']), 3, embeddings_regularizer=l2(), name="genre_embedding")(gender)
        # gender_layer = Flatten()(gender_embedding)

        # G_embedding = Flatten()(G_embedding)
        p_embedding = Flatten()(p_embedding)
        Q_embedding = Flatten()(Q_embedding)
        # gender_bias = Flatten()(gender_bias)
        # item_bias = Flatten()(item_bias)

        # Concatenate layers
        R = Concatenate()([p_embedding, Q_embedding, genre_layer])

        # Neural network
        R = Dense(2048)(R)
        R = Activation('relu')(R)
        R = Dense(256)(R)
        R = Activation('linear')(R)
        R = Dense(1)(R)


        model = Model(inputs=[user_input, movie_input, genre_input], outputs=R)

        # 이상한 점 : 0,1로 binary인데 loss 값을 BinaryCrossentropy로 주지 않음
        model.compile(
        loss= 'mse',
        # optimizer=SGD(),
        optimizer=Adam(),
        metrics='acc'
        )

        # return model, raw_data, ratings_train, ratings_test
        return model, raw_data, ratings_train, ratings_test, genre_arr_train, genre_arr_test

In [ ]:
def model_array_train(model, epoch, recommend_model_number = 1,train = False):
    if train==1:
        model.fit(
                x=[ratings_train.userId.values, ratings_train.movieId.values, genre_arr_train],
                y=ratings_train.rating.values,
                epochs=epoch,
                batch_size=512,
                validation_split = 0.2,
                verbose=1
                )
        model.save("/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220922_3D_Embedding_model저장 및 비교/reco_model_array_{}.h5".format(recommend_model_number))
        return model
    elif train==2:
        model.fit(
                x=[gender_arr_train, ratings_train.movieId.values, genre_arr_train],
                y=ratings_train.rating.values,
                epochs=epoch,
                batch_size=512,
                validation_split = 0.2,
                verbose=1
                )
        model.save("/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220922_3D_Embedding_model저장 및 비교/20221012onehot_reco_array_model_{}.h5".format(recommend_model_number))
        return model
    elif train==3:
        model.fit(
                x=[gender_arr_train, ratings_train.movieId.values],
                y=ratings_train.rating.values,
                epochs=epoch,
                batch_size=512,
                validation_split = 0.2,
                verbose=1
                )
        return model
    elif train==4:
        model.fit(
                x=[age_arr_train, ratings_train.movieId.values, genre_arr_train],
                y=ratings_train.rating.values,
                epochs=epoch,
                batch_size=512,
                validation_split = 0.2,
                verbose=1
                )
        return model   
    elif train==5:
        model.fit(
                x=[age_arr_train, ratings_train.movieId.values],
                y=ratings_train.rating.values,
                epochs=epoch,
                batch_size=512,
                validation_split = 0.2,
                verbose=1
                )
        return model    
 

    else:
        path = "/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220922_3D_Embedding_model저장 및 비교/20221012onehot_reco_array_model_{}.h5".format(recommend_model_number)
        model = tf.keras.models.load_model(path)
        return model


In [ ]:
def recommendation(test_data, gender, userId):
    user_ids = test_data.userId.values
    movie_ids = test_data.movieId.values
    ## labeling - feature 별 추가
    # gender_ids = ratings_test.gender.values
    # genre_ids = ratings_test.genres.values

    ## onehot-feature별 추가
    # gender_ids = np.array([np.array(t) for t in gender_arr_test])
    genre_ids = np.array([np.array(t) for t in genre_arr_test])
    # age_ids = np.array([np.array(t) for t in age_arr_test])
    

    # y_pred = model.predict([user_ids, movie_ids])
    y_pred = model.predict([user_ids, movie_ids, genre_ids])
    y_pred = np.ravel(y_pred, order='C')
    y_true = np.array(test_data.rating)

    # 성능평가 RMSE
    def RMSE(y_true, y_pred):
        return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

    rmse_score = 'Recommendation RMSE: {:.4f}'.format(RMSE(y_true, y_pred))

    # 추천 TOP 10
    movie_list = test_data.movieId.unique().astype(int).tolist()
    # movie_list_df = pd.DataFrame(movie_list, columns=['movieId'])
    # movie_list_df['userId'] = userId
    test_data['genre_2'] = genre_ids.tolist()
    user_data_df = pd.DataFrame()
    for i in movie_list:
        test_data_2 = test_data[test_data['userId'] == i].head(1)
        user_data_df = pd.concat([user_data_df, test_data_2], axis=0)

    user_data_df = user_data_df[['movieId', 'genre_2']]
    user_data_df['userId'] = userId


    user_data_df['pred'] = model.predict([user_data_df.userId.values, user_data_df.movieId.values, np.array([np.array(t) for t in user_data_df['genre_2']])])

    recommend_top10_list = user_data_df.sort_values(by='pred', ascending=False).head(10)['movieId'].tolist()
    recommend_top10_df = pd.DataFrame()
    for i in recommend_top10_list:
        tmp = test_data[test_data['movieId']==i][:1]
        recommend_top10_df = pd.concat([recommend_top10_df, tmp], axis=0).drop(columns=['userId','rating','gender','age','occupation'])
        # recommend_top10_df = pd.concat([recommend_top10_df, tmp], axis=0).drop(columns=['userId','rating','gender','age','occupation', 'gender_2'])

    return rmse_score, recommend_top10_df

In [ ]:
lens_df = load_data('lens_df_1M')

In [ ]:
lens_df

,movieId,title,genres,count,userId,rating,gender,age,occupation
0,748,"Shining, The (1980)",Horror,1087,959,0,M,35,14
1,51,"Relic, The (1997)",Horror,237,990,1,M,18,4
2,2134,"Fog, The (1980)",Horror,170,1416,0,M,25,17
3,1845,"Wolf Man, The (1941)",Horror,134,916,1,M,18,4
4,2961,Love Is a Many-Splendored Thing (1955),Romance,50,209,0,M,25,4
...,...,...,...,...,...,...,...,...,...
43211,2091,Metro (1997),Action,100,326,0,M,25,0
43212,748,"Shining, The (1980)",Horror,1087,304,1,M,25,3
43213,1076,Hoop Dreams (1994),Documentary,716,2988,1,F,25,15
43214,865,Double Team (1997),Action,48,11,0,M,25,3


In [ ]:
# 2. data 정제하기
# data one-hot encoding
# genre_array = genre_onehot(lens_df)
genre_array = genre_4_onehot(lens_df)
# gender_array = gender_onehot(lens_df)
# age_array = age_onehot(lens_df)

In [ ]:
# 3. neural network model 
# model, ratings_train, ratings_test, genre_arr_train, genre_arr_test, gender_arr_train, gender_arr_test, age_arr_train, age_arr_test = deep_array_model(lens_df, genre_array, age_array, gender_array, TRAIN_SIZE=0.8, K=200)
model, raw_data, ratings_train, ratings_test, genre_arr_train, genre_arr_test = deep_array_model(lens_df, genre_array, TRAIN_SIZE=0.8, K=200)
# model, raw_data, ratings_train, ratings_test = deep_array_model(lens_df, TRAIN_SIZE=0.8, K=200)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
movie_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
genre_input (InputLayer)        [(None, 4)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 200)       1207200     user_input[0][0]                 
______________________________________________________________________________________________

2022-10-16 13:27:19.185709: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# 4. train model / load model.save
# train =1 (학습함) / train = 0 (load model)
# 
epoch = 5
recommend_model_number = 1
# model = model_train(model, epoch, recommend_model_number, train=True)
model = model_array_train(model, epoch, 1, 1)


Epoch 1/5


2022-10-16 13:27:19.465912: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


55/55 [==============================] - 7s 106ms/step - loss: 4.4490 - acc: 0.6360 - val_loss: 0.2933 - val_acc: 0.7005
Epoch 2/5
55/55 [==============================] - 6s 112ms/step - loss: 0.2298 - acc: 0.7126 - val_loss: 0.2241 - val_acc: 0.7121
Epoch 3/5
55/55 [==============================] - 6s 101ms/step - loss: 0.2160 - acc: 0.7216 - val_loss: 0.2129 - val_acc: 0.7248
Epoch 4/5
55/55 [==============================] - 6s 102ms/step - loss: 0.2100 - acc: 0.7279 - val_loss: 0.2092 - val_acc: 0.7267
Epoch 5/5
55/55 [==============================] - 6s 101ms/step - loss: 0.2062 - acc: 0.7303 - val_loss: 0.2137 - val_acc: 0.7174


In [ ]:
# 5. 실제 rating 값, 예측 rating 값 가져오기 + RMSE + 추천결과까지 print
movieid = int(input('movieid를 입력하세요'))
rmse, recommend_top10_df = recommendation(ratings_test, 'M', movieid)
print(rmse)
print(recommend_top10_df)

movieid를 입력하세요 15


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Recommendation RMSE: 0.4346
       movieId                                              title  \
34682     1126                          When We Were Kings (1996)   
34626     1909                               War Room, The (1993)   
34626     1909                               War Room, The (1993)   
34641     1076                                 Hoop Dreams (1994)   
34838     2833  Wonderful, Horrible Life of Leni Riefenstahl, ...   
34735     1680                           Stop Making Sense (1984)   
34823      534        Decline of Western Civilization, The (1981)   
34823      534        Decline of Western Civilization, The (1981)   
34794     2649                        Hands on a Hard Body (1996)   
34596     1141                              American Movie (1999)   

            genres  count       genre_2  
34682  Documentary    277  [0, 1, 0, 0]  
34626  Documentary    132  [0, 1, 0, 0]  
34626  Documentary    132  [0, 1, 0, 0]  
34641  Documentary    716  [0, 1, 0, 0]  
34

In [ ]:
def extract_weights(name, model):
    # Extract weights
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    
    # Normalize
    # Normaliza 과정 후 tsv가 embedding projection에 안먹힐 때 normalization 과정을 스킵하고 넣으면 들어가짐
    # 오류 원인... float32로 들어갈 수 없는 작은값을 표현하려고한 np.nan_to_num과정에서 오류가 난것으로 보임
    # weights = (weights / np.linalg.norm(weights, axis = 1)).reshape((-1, 1))
    # weights_g = np.nan_to_num(weights, copy=False)

    df = pd.DataFrame(weights)

    # # movie
    df['movie'] = raw_data['movieId'].unique().tolist()
    df = df.sort_values(by='movie')
    df = df.set_index('movie')
    weights_g = df.to_numpy()
    # df.to_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220928_scatter_user_mapping/db/20221011_movie_genre_scateer.csv')/
    # df.to_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220928_scatter_user_mapping/db/20221012_6pm_user_movie_scatter01.tsv', index=False, sep='\t', header=False)

    return weights_g

def extract_weights(name, model):
    # Extract weights
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    


    df = pd.DataFrame(weights)

    user
    df['user'] = raw_data['userId'].unique().tolist()
    df = df.sort_values(by='user')
    df = df.set_index('user')
    weights_g = df.to_numpy()

    return weights_g

In [ ]:
def k_means(weight, k):
    df = pd.DataFrame(weight).astype('float64')
    scaler = MinMaxScaler()
    data_scale = scaler.fit_transform(df)

    from sklearn.cluster import KMeans
    # 그룹 수, random_state 설정
    k_model = KMeans(n_clusters = k, random_state = 10, n_init=10, max_iter=1000, tol=1e-9, verbose=1 )

    # 정규화된 데이터에 학습
    k_model.fit(data_scale)

    # 클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
    # y_clusters = k_model.fit_predict(data_scale)
    df['cluster'] = k_model.fit_predict(data_scale)
    

    # user_df = pd.DataFrame(raw_data.userId.unique(), columns=['userId'])
    movie_df = pd.DataFrame(lens_df.movieId.unique(), columns=['movieId'])
    # merge_df = pd.concat([df, user_df], axis=1)
    merge_df = pd.concat([df, movie_df], axis=1)
    # merge_scatter = merge_df.drop(columns=['cluster', 'userId'], axis=1)
    merge_scatter = merge_df.drop(columns=['cluster', 'movieId'], axis=1)
    import pdb;pdb.set_trace()
    # merge_scatter.to_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20221013_scatter_user_mapping/db/20201011_genre_embedding_3.csv')
    merge_scatter.to_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220928_scatter_user_mapping/db/20221014_onehot_movie_genre_embedding_10_clustering_scatter.tsv', index=False, header=False, sep='\t')
    merge_label = merge_df[['cluster','movieId']]
    merge_label.to_csv('/home/juno/workspace/user_collaborative_filtering/deeplearning_recomend/yclass/20220928_scatter_user_mapping/db/20221014_onehot_movie_genre_embedding_10_cluster.tsv', index=False, sep='\t')

    return merge_df

In [ ]:
# weights_class = extract_weights('user_embedding', model)
weights_class = extract_weights('movie_embedding', model)


NameError: name 'user' is not defined

In [ ]:

df = k_means(weights_class, 10)

In [ ]:
%debug (k_means)